#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [2]:
import pandas as pd
df = pd.read_csv('../data/supermarket_sales.csv')

1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [18]:
branch_income = df.groupby('Branch')['gross income'].sum()
branch_income

Branch
A    5057.1605
B    5057.0320
C    5265.1765
Name: gross income, dtype: float64

In [20]:
print(f"Найприбутковіший філіал: {branch_income.idxmax()} і його сумарний прибуток складає {branch_income.max():.2f}")

Найприбутковіший філіал: C і його сумарний прибуток складає 5265.18


2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

In [26]:
df.groupby(['Branch', 'City'])['gross income'].sum()

Branch  City     
A       Yangon       5057.1605
B       Mandalay     5057.0320
C       Naypyitaw    5265.1765
Name: gross income, dtype: float64

In [34]:
print(f"Місто з найвищим сумарним прибутком: {df.loc[df['gross income'].idxmax(), 'City']}")

Місто з найвищим сумарним прибутком: Naypyitaw


3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [46]:
invoices_by_category = df.pivot_table(
    index='Branch',                  
    columns='Product line',
    values='Invoice ID',       
    aggfunc='count',               
    fill_value=0             
)

In [48]:
invoices_by_category

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Branch,,,,,,
A,60,51,58,47,65,59
B,55,62,50,53,50,62
C,55,65,66,52,45,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Використовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [68]:
branch_max_invoices = invoices_by_category['Electronic accessories'].idxmax()
max_invoices = invoices_by_category['Electronic accessories'].max()

In [70]:
print(f"Філіал з найбільшою кількістю інвойсів для 'Electronic accessories': {branch_max_invoices}, їх кількість становить {max_invoices}")

Філіал з найбільшою кількістю інвойсів для 'Electronic accessories': A, їх кількість становить 60


4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


1. Чи залежать продажі від днів тижня і часу доби?
2. Чи існує звязок між способом оплати покупок і кількістю здійснених покупок? Які види покупок оплачують тим чи іншим способом частіше?
3. Як тип клієнту частіше здійснює покупки кожної лінійки продукції?

In [174]:
# 1 питання(гіпотеза) можемо побачити в який час доби і дня тижня купують товарів на найбільші суми
sales_by_day_time = df.groupby(['Date', 'Time']).agg({
    'Total': 'sum'
}).reset_index()
df['Date'] = pd.to_datetime(df['Date'])
df['Day of Week'] = df['Date'].dt.day_name()

In [176]:
def get_hour(time_str):
    hour_str = time_str.split(':')[0]
    return int(hour_str)
df['Hour'] = df['Time'].apply(get_hour)

sales_by_day_hour = df.groupby(['Day of Week', 'Hour']).agg({
    'Total': 'sum'
}).reset_index()

In [178]:
sales_by_day_hour 

,Day of Week,Hour,Total
0,Friday,10,5671.6905
1,Friday,11,2649.2865
2,Friday,12,2159.2410
3,Friday,13,6824.3070
4,Friday,14,5166.2730
...,...,...,...
72,Wednesday,16,3229.6215
73,Wednesday,17,3101.2695
74,Wednesday,18,4233.7680
75,Wednesday,19,4160.1735


In [140]:
# 2 питання(гіпотеза) можемо побачити яким способом частіше оплачують певні види покупок, також можна подивитись ситуацію по містам (додавши колонку City)
payment_quantity_summary = df.groupby(['Payment', 'Product line']).agg({
    'Quantity': 'sum'
}).reset_index()
payment_quantity_summary 

,Payment,Product line,Quantity
0,Cash,Electronic accessories,398
1,Cash,Fashion accessories,280
2,Cash,Food and beverages,295
3,Cash,Health and beauty,293
4,Cash,Home and lifestyle,306
5,Cash,Sports and travel,324
6,Credit card,Electronic accessories,262
7,Credit card,Fashion accessories,293
8,Credit card,Food and beverages,335
9,Credit card,Health and beauty,274


In [148]:
max_quantity_summary = payment_quantity_summary.loc[
    payment_quantity_summary.groupby('Payment')['Quantity'].idxmax()
].reset_index(drop=True)
max_quantity_summary
# тут бачимо яким зі способів найчастіше оплачують певну лінійку товарів, тобто готівкою електронні аксесуари, кредиткою - їжу та напої, е-гаманцем - товари для дому і життя

,Payment,Product line,Quantity
0,Cash,Electronic accessories,398
1,Credit card,Food and beverages,335
2,Ewallet,Home and lifestyle,346


In [100]:
# 3 питання(гіпотеза)
purchase_by_customer_type = df.groupby(['Customer type', 'Product line']).agg({
    'Total': 'mean',   # тут можна взяти і min/max, щоб прослідкувати тенденції щодо мінімальних/максимальних cум покупок
    'Quantity': 'sum'
}).reset_index()
purchase_by_customer_type # тут бачимо, дані по кожному виду покупця

,Customer type,Product line,Total,Quantity
0,Member,Electronic accessories,314.083269,429
1,Member,Fashion accessories,306.092581,439
2,Member,Food and beverages,333.591702,506
3,Member,Health and beauty,353.849856,428
4,Member,Home and lifestyle,337.084663,490
5,Member,Sports and travel,324.532190,493
6,Normal,Electronic accessories,324.337353,542
7,Normal,Fashion accessories,304.151446,463
8,Normal,Food and beverages,309.840300,446
9,Normal,Health and beauty,295.730373,426


Як ці питання і відповіді на них можна використати в бізнесі при прийнятті рішень?
1. Можна робити знижки "щасливі години" в певний час тижня, в подальшому цей аналіз можна розширити до локації магазину (City) і групи товару (Product line), аби "щасливі години" були на певні групи товару і в певних локаціях. це дозволить додатково простимулювати продажі та збільшити прибуток, підвищити лояльність покупців.
2. Так як вцілому спостерігається, що готівкою оплачують значно менше, ніж картками чи електронним гаманцем, то для магазинів можна ввести систему кешбеків на оплату карткою/електронно, щоб більше користувачів здійснювало безготівкову оплату, таким чином зменшити обіг готівки (зменшаться проблеми з видачею решти, недостачами, витратами часу на інкасацію коштів, зменшенням кількості злодіїв, які крадуть гроші з кас)ю
3. Тут можна проспостерігати як працює взагалі членська система і подумати як збільшити лояльність наявних покупців з членською картою та як їх віддячити за приєднання. Наприклад, додати бонуси за покупку товарів, які купуються найменше, пропонувати карту членства безкоштовно за покупку певних категорій товарів